In [1]:
import tensorflow as tf
import pathlib
# from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input


In [2]:
#path dataset dan parameter
base_dir = pathlib.Path(".") 
data_split_dir = base_dir / 'dataset_final'

img_height = 224
img_width = 224
batch_size = 32
seed = 42
autotune = tf.data.AUTOTUNE

NUM_CLASSES = 4  # Jumlah kelas untuk klasifikasi aflatoksin

In [38]:
# import shutil
# from sklearn.model_selection import train_test_split

# #path dataset dan parameter
# base_dir = pathlib.Path(".") 
# data_dir = base_dir / "data valid merah"
# data_split_dir =base_dir / "data_split_merah"
# # data_dir = pathlib.Path(r"D:\SKRIPSI\skripsi\data valid merah")
# # data_split_dir = pathlib.Path(r"D:\SKRIPSI\skripsi\data_split_merah")
# img_height = 224
# img_width = 224
# batch_size = 32
# seed = 42
# autotune = tf.data.AUTOTUNE

# # Script pembagian data: 70% train, 20% validasi, 10% test
# print("Step 1: Splitting data...")
# def split_data_folder(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
#     source_dir = pathlib.Path(source_dir)
#     dest_dir = pathlib.Path(dest_dir)
#     dest_dir.mkdir(parents=True, exist_ok=True)

#     for class_folder in source_dir.iterdir():
#         if not class_folder.is_dir():
#             continue
#         files = list(class_folder.glob('*.png'))
#         if len(files) == 0:
#             continue

#         train_files, temp_files = train_test_split(files, train_size=train_ratio, random_state=42)
#         val_size = val_ratio / (val_ratio + test_ratio)
#         val_files, test_files = train_test_split(temp_files, train_size=val_size, random_state=42)

#         for split_name, split_files in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
#             split_class_dir = dest_dir / split_name / class_folder.name
#             split_class_dir.mkdir(parents=True, exist_ok=True)
#             for f in split_files:
#                 shutil.copy(str(f), str(split_class_dir / f.name))

# # Contoh penggunaan:
# split_data_folder(
#     source_dir=data_dir,
#     # source_dir=r'D:\SKRIPSI\skripsi\data valid merah balanced',
#     dest_dir=data_split_dir,
#     train_ratio=0.7, val_ratio=0.15, test_ratio=0.15
# )

In [3]:
# --- HEADER UTAMA ---
print("==================PROSES PEMUATAN DATASET==================")
# 1. Load Train Dataset
print(f"\n[1/3] Memuat {data_split_dir / 'train'}")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'train',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

# 2. Load Validation Dataset
print(f"\n[2/3] Memuat {data_split_dir / 'val'}")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'val',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False, 
    seed=seed
)

# 3. Load Test Dataset
print(f"\n[3/3] Memuat {data_split_dir / 'test'}")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'test',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

# --- Informasi Kelas & Mapping ---
class_names = train_ds.class_names

print(f"\n==================INFORMASI KELAS & MAPPING==================")
print(f"Kelas ditemukan: {class_names}")

# Menampilkan Tabel Mapping yang Rapi
print(f"\n{'Index Model':<15} | {'Nama Folder':<15} | {'Nilai PPB (Target)':<20}")
print("-" * 60)
for idx, name in enumerate(class_names):
    print(f"{idx:<15} | {name:<15} | {name:<25}")

print(f"\nSelesai. Dataset siap digunakan.")

==================PROSES PEMUATAN DATASET==================

[1/3] Memuat dataset_final\train
Found 1400 files belonging to 4 classes.

[2/3] Memuat dataset_final\val
Found 131 files belonging to 4 classes.

[3/3] Memuat dataset_final\test
Found 132 files belonging to 4 classes.

==================INFORMASI KELAS & MAPPING==================
Kelas ditemukan: ['1', '2', '3', '4']

Index Model     | Nama Folder     | Nilai PPB (Target)  
------------------------------------------------------------
0               | 1               | 1                        
1               | 2               | 2                        
2               | 3               | 3                        
3               | 4               | 4                        

Selesai. Dataset siap digunakan.


In [4]:
print("Step 2: Setting up data augmentation...")

# Data augmentation using Keras layers (TensorFlow 2.x compatible)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.028),  # 0.1 radians ≈ 5.7 degrees (smaller rotation)
    # tf.keras.layers.RandomBrightness(0.1),
    # tf.keras.layers.RandomContrast(0.1)
], name="data_augmentation")

print("✓ RandomFlip (horizontal)")
print("✓ RandomRotation (±10°)")


Step 2: Setting up data augmentation...
✓ RandomFlip (horizontal)
✓ RandomRotation (±10°)


In [5]:
# =============================================================================
# CELL 5: FUNGSI PREPROCESSING
# =============================================================================
#
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  🔄 PERUBAHAN: Fungsi preprocess_input yang dipanggil berbeda             ║
# ║                                                                           ║
# ║  Meskipun KODE-nya SAMA, PERILAKU-nya BERBEDA karena:                     ║
# ║  - Di Cell 1, kita import dari 'efficientnet' bukan 'resnet50'            ║
# ║  - Jadi preprocess_input() di sini adalah versi EfficientNet              ║
# ║                                                                           ║
# ║  PERBEDAAN NORMALISASI:                                                   ║
# ║  ┌─────────────────────────────────────────────────────────────────────┐  ║
# ║  │ ResNet50 (mode "caffe"):                                            │  ║
# ║  │   1. Konversi RGB → BGR                                             │  ║
# ║  │   2. Kurangi mean ImageNet: [103.939, 116.779, 123.68]              │  ║
# ║  │   3. Output range: sekitar [-128, 128]                              │  ║
# ║  ├─────────────────────────────────────────────────────────────────────┤  ║
# ║  │ EfficientNet (mode "torch"):                                        │  ║
# ║  │   1. Bagi dengan 255 → range [0, 1]                                 │  ║
# ║  │   2. Normalize: (x - mean) / std                                    │  ║
# ║  │      mean = [0.485, 0.456, 0.406]                                   │  ║
# ║  │      std = [0.229, 0.224, 0.225]                                    │  ║
# ║  │   3. Output range: sekitar [-2.1, 2.6]                              │  ║
# ║  │   4. Tetap RGB (tidak dikonversi ke BGR)                            │  ║
# ║  └─────────────────────────────────────────────────────────────────────┘  ║
# ╚═══════════════════════════════════════════════════════════════════════════╝
# =============================================================================

def preprocess_for_efficientnet(images, labels, training=False):
    """
    Preprocessing untuk EfficientNet.
    
    CATATAN: Meskipun struktur kode sama dengan versi ResNet50,
    fungsi preprocess_input() yang dipanggil adalah versi EfficientNet
    (karena import di Cell 1).
    
    Args:
        images: Batch gambar dengan nilai pixel 0-255
        labels: One-hot encoded labels
        training: Boolean, True untuk training (apply augmentation)
    
    Returns:
        Tuple (preprocessed_images, labels)
    """
    # Cast ke float32
    images = tf.cast(images, tf.float32)
    
    # Terapkan augmentasi HANYA saat training
    # (hanya flip dan rotation - TIDAK ADA brightness/contrast)
    if training:
        images = data_augmentation(images, training=True)
    
    # ⬇️⬇️⬇️ INI YANG BERBEDA PERILAKUNYA ⬇️⬇️⬇️
    # Meskipun kode sama, fungsi ini adalah versi EfficientNet
    # karena import dari tensorflow.keras.applications.efficientnet
    images = preprocess_input(images)
    # ⬆️⬆️⬆️ INI YANG BERBEDA PERILAKUNYA ⬆️⬆️⬆️
    
    return images, labels

In [6]:
# =============================================================================
# CELL 6: TERAPKAN PREPROCESSING & OPTIMASI PIPELINE
# =============================================================================
#
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║  🔄 PERUBAHAN: Menambahkan .cache() dan .prefetch()                       ║
# ║                                                                           ║
# ║  SEBELUM (ResNet50):                                                      ║
# ║  train_ds = train_ds.map(...)                                             ║
# ║                                                                           ║
# ║  SESUDAH (EfficientNet):                                                  ║
# ║  train_ds = train_ds.map(...).cache().prefetch(...)                       ║
# ║                                                                           ║
# ║  PENJELASAN:                                                              ║
# ║  • .cache() = simpan data di memory setelah load pertama                  ║
# ║               Epoch 2+ akan lebih cepat karena tidak perlu baca disk      ║
# ║  • .prefetch() = load batch berikutnya sementara GPU proses batch saat ini║
# ║                  Menghilangkan bottleneck I/O                             ║
# ╚═══════════════════════════════════════════════════════════════════════════╝
# =============================================================================

print("Menerapkan preprocessing ke dataset...")

# Training Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
train_ds = train_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=True),
    num_parallel_calls=autotune
).cache().prefetch(buffer_size=autotune)

print("✓ Training dataset: augmentation ON, cache ON, prefetch ON")

# Validation Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
val_ds = val_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=False),
    num_parallel_calls=autotune
).cache().prefetch(buffer_size=autotune)

print("✓ Validation dataset: augmentation OFF, cache ON, prefetch ON")

# Test Dataset
# ⬇️ PERUBAHAN: Menambahkan .cache().prefetch() ⬇️
test_ds = test_ds.map(
    lambda x, y: preprocess_for_efficientnet(x, y, training=False),
    num_parallel_calls=autotune
).cache().prefetch(buffer_size=autotune)

print("✓ Test dataset: augmentation OFF, cache ON, prefetch ON")

print("\n" + "="*60)
print("PREPROCESSING SELESAI")
print("="*60)
print("\nDataset siap digunakan untuk training EfficientNet-B0!")

Menerapkan preprocessing ke dataset...
✓ Training dataset: augmentation ON, cache ON, prefetch ON
✓ Validation dataset: augmentation OFF, cache ON, prefetch ON
✓ Test dataset: augmentation OFF, cache ON, prefetch ON

PREPROCESSING SELESAI

Dataset siap digunakan untuk training EfficientNet-B0!


In [7]:
# =============================================================================
# CELL 7: VERIFIKASI PREPROCESSING
# =============================================================================
# Cell ini memverifikasi bahwa preprocessing EfficientNet bekerja dengan benar
# Range nilai yang diharapkan: sekitar [-2.1, 2.6] (berbeda dengan ResNet50!)
# =============================================================================

print("Verifikasi preprocessing...")

for images, labels in train_ds.take(1):
    print(f"\nBatch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"\nStatistik pixel setelah preprocessing:")
    print(f"  - Min: {tf.reduce_min(images).numpy():.4f}")
    print(f"  - Max: {tf.reduce_max(images).numpy():.4f}")
    print(f"  - Mean: {tf.reduce_mean(images).numpy():.4f}")
    print(f"  - Std: {tf.math.reduce_std(images).numpy():.4f}")
    
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    
    # EfficientNet preprocess_input menghasilkan nilai dalam range sekitar [-2.1, 2.6]
    # (berbeda dengan ResNet50 yang ~[-128, 128])
    if -3 < min_val < 0 and 0 < max_val < 3:
        print("\n✅ Range nilai sesuai dengan ekspektasi EfficientNet")
        print("   (EfficientNet: ~[-2.1, 2.6], ResNet50: ~[-128, 128])")
    elif min_val < -50 or max_val > 50:
        print("\n⚠️ WARNING: Range nilai terlihat seperti ResNet50!")
        print("   Pastikan import dari efficientnet, bukan resnet50")
    else:
        print("\n⚠️ Range nilai tidak sesuai ekspektasi")

Verifikasi preprocessing...

Batch shape: (32, 224, 224, 3)
Labels shape: (32, 4)

Statistik pixel setelah preprocessing:
  - Min: 0.0000
  - Max: 209.3575
  - Mean: 0.1618
  - Std: 2.7151

⚠️ WARNING: Range nilai terlihat seperti ResNet50!
   Pastikan import dari efficientnet, bukan resnet50
